In [1]:
from helper import get_connected, catch
from snp_variables import fspath

from ib_insync import *
util.startLoop()

ib = get_connected('snp', 'live')

util.logToFile(fspath+'_errors.log')  # create log file
with open(fspath+'_errors.log', 'w'): # clear the previous log
    pass

In [ ]:
# p_snps.py
import pandas as pd
from snp_variables import blk
from ib_insync import *

def p_snps(ib):
    '''Pickles snps underlying (1 minute)
    Arg: (ib) as connection object
    Returns: Dataframe of symbol, lot, margin with underlying info'''

    # exclusion list
    excl = ['VXX','P', 'TSRO']

    # Download cboe weeklies to a dataframe
    dls = "http://www.cboe.com/publish/weelkysmf/weeklysmf.xls"

    #     snp500 = list(pd.read_html('https://en.wikipedia.org/wiki/List_of_S%26P_500_companies')[0][1:].loc[:, 1])

    snp100 = pd.read_html('https://en.wikipedia.org/wiki/S%26P_100')[1][1:].loc[:, 0]
    snp100 = [s.replace('.', ' ') if '.' in s else s  for s in snp100] # without dot in symbol
    # read from row no 11, dropna and reset index
    df_cboe = pd.read_excel(dls, header=12, 
                            usecols=[0,2,3]).loc[11:, :]\
                            .dropna(axis=0)\
                            .reset_index(drop=True)

    # remove column names white-spaces and remap to IBKR
    df_cboe.columns = df_cboe.columns.str.replace(' ', '')

    # remove '/' for IBKR
    df_cboe.Ticker = df_cboe.Ticker.str.replace('/', ' ', regex=False)

    # make symbols
    symbols = {s for s in df_cboe.Ticker if s not in excl if s in snp100}
    stocks = [Stock(symbol=s, exchange='SMART', currency='USD') for s in symbols]

    stkblks = [stocks[i: i+blk] for i in range(0, len(stocks), blk)] # blocks of stocks

    # qualify the contracts
    contracts = [ib.qualifyContracts(*s) for s in stkblks]
    contracts = [contract for subl in contracts for contract in subl]
    qcs_dict = {q.symbol: q for q in contracts}

    # get the margins
    orders = [Order(action='SELL', orderType='MKT', totalQuantity=100, whatIf=True)]*len(contracts)
    margins = [ib.whatIfOrder(c, o).initMarginChange for c, o in zip(contracts, orders)]
    m_dict = {s.symbol:m for s, m in zip(contracts, margins)}

    # get the undPrices
    tickers = ib.reqTickers(*contracts)
    undPrices = {t.contract.symbol: t.marketPrice() for t in tickers} # {symbol: undPrice}

    qcs_dict = {q.symbol: q for q in contracts}

    # contracts, lots, margins, undPrices dataframe
    df_clmu = pd.DataFrame.from_dict(qcs_dict, orient='index', columns=['contract']).\
             join(pd.DataFrame.from_dict(m_dict, orient='index', columns=['margin'])).\
             join(pd.DataFrame.from_dict(undPrices, orient='index', columns=['undPrice']))

    df_clmu = df_clmu.assign(lot=100)

    df_clmu = df_clmu.assign(margin=abs(pd.to_numeric(df_clmu.margin)).astype('int'))
    
    df_clmu.columns=['contract', 'margin', 'undPrice', 'lot']
    df_clmu.to_pickle(fspath+'_lot_margin.pickle')
    
    return df_clmu

In [ ]:
# p_snpopts.py
import numpy as np
import pandas as pd
from itertools import product, repeat

from helper import get_dte, get_maxfallrise, get_rollingmax_std
from snp_variables import *

def p_snpopts(ib, undContract, undPrice, lotsize=100):
    '''Pickles the option chains
    Args:
        (ib) ib connection as object
        (undContract) underlying contract as object
        (undPrice) underlying contract price as float
        (margin) margin of undContract (used as a surrogate for option)
        (lotsize) lot-size as float is 100
        <fspath> file path to store snp options'''
    
    symbol = undContract.symbol
    
    chains = ib.reqSecDefOptParams(underlyingSymbol = symbol,
                         futFopExchange = '',
                         underlyingSecType = undContract.secType,
                         underlyingConId= undContract.conId)

    xs = [set(product(c.expirations, c.strikes)) for c in chains if c.exchange == exchange]

    expirations = [i[0] for j in xs for i in j]
    strikes = [i[1] for j in xs for i in j]
    dflength = len(expirations)

    #...first df with symbol, strike and expiry
    df1 = pd.DataFrame({'cid': pd.Series(np.repeat(undContract.conId,dflength)), 
                  'symbol': pd.Series(np.repeat(symbol,dflength)),
                  'expiration': expirations,
                  'strike': strikes,
                  'dte': [get_dte(e) for e in expirations],
                  'undPrice': pd.Series(np.repeat(undPrice,dflength))})

    df2 = df1[(df1.dte > mindte) & (df1.dte < maxdte)].reset_index(drop=True)  # limiting dtes

    dtes = df2.dte.unique().tolist()

    #...get the max fall / rise for puts / calls
    maxFallRise = {d: get_maxfallrise(ib, c, d) for c, d in zip(repeat(undContract), dtes)}

    df3 = df2.join(pd.DataFrame(df2.dte.map(maxFallRise).tolist(), index=df2.index, columns=['lo52', 'hi52', 'Fall', 'Rise']))

    df4 = df3.assign(loFall = df3.undPrice-df3.Fall, hiRise = df3.undPrice+df3.Rise)

    std = {d: get_rollingmax_std(ib, c, d) for c, d in zip(repeat(undContract), dtes)}

    df4['std3'] = df4.dte.map(std)*minstdmult

    df4['loStd3'] = df4.undPrice - df4.std3
    df4['hiStd3'] = df4.undPrice + df4.std3

    # flter puts and calls by standard deviation
    df_puts = df4[df4.strike < df4.loStd3]
    df_calls = df4[df4.strike > df4.hiStd3]

    # with rights
    df_puts = df_puts.assign(right='P')
    df_calls = df_calls.assign(right='C')

    # qualify the options
    df_opt1 = pd.concat([df_puts, df_calls]).reset_index()

    optipl = [Option(s, e, k, r, exchange) for s, e, k, r in zip(df_opt1.symbol, df_opt1.expiration, df_opt1.strike, df_opt1.right)]

    optblks = [optipl[i: i+blk] for i in range(0, len(optipl), blk)] # blocks of optipl

    # qualify the contracts
    contracts = [ib.qualifyContracts(*s) for s in optblks]
    
    try:
        [z for x in contracts for y in x for z in y] # If an Option is available, this will fail
        if not [z for x in contracts for y in x for z in y]:  # Check if there is anything in the list
            return None  # The list is empty in 3 levels!
    except TypeError as e:
        pass

    q_opt = [d for c in contracts for d in c]

    opt_iDict = {c.conId: c for c in q_opt}

    df_opt1 = util.df(q_opt).loc[:, ['conId', 'symbol', 'lastTradeDateOrContractMonth', 'strike', 'right']]

    df_opt1 = df_opt1.rename(columns={'lastTradeDateOrContractMonth': 'expiration', 'conId': 'optId'})

    opt_tickers = ib.reqTickers(*q_opt)

    # get the price
    df_opt1 = df_opt1.assign(optPrice = [t.marketPrice() for t in opt_tickers])
    df_opt1 = df_opt1[df_opt1.optPrice > 0.0]

    # get the margins
    opts = df_opt1.optId.map(opt_iDict)
    orders = [Order(action='SELL', orderType='MKT', totalQuantity=1, whatIf=True)]*len(opts)
    margins = [ib.whatIfOrder(c, o).initMarginChange for c, o in zip(opts, orders)]
    df_opt1 = df_opt1.assign(optMargin = margins)
    df_opt1 = df_opt1.assign(optMargin = pd.to_numeric(df_opt1.optMargin)) # convert to numeric

    cols=['symbol', 'expiration', 'strike']
    df_opt2 = pd.merge(df4, df_opt1, on=cols).drop('cid', 1).reset_index(drop=True)

    # Get lotsize and margin for the underlying symbol
    df_opt2 = df_opt2.assign(lotsize = lotsize)
    
#     # get the margins
#     orders = [Order(action='SELL', orderType='MKT', totalQuantity=100, whatIf=True)]*len(q_opt)
#     margins = [ib.whatIfOrder(c, o).initMarginChange for c, o in zip(q_opt, orders)]
    
#     df_opt2 = df_opt2.assign(optMargin = margins)

#     opt_contracts = [opt_iDict[i] for i in df_opt2.optId]

    df_opt2 = df_opt2.assign(rom=df_opt2.optPrice*df_opt2.lotsize/df_opt2.optMargin*252/df_opt2.dte).sort_values('rom', ascending=False)
    
    # arrange the columns
    cols = ['optId', 'symbol', 'right', 'expiration', 'dte', 'strike', 'undPrice', 
            'lo52', 'hi52', 'Fall', 'Rise', 'loFall', 'hiRise', 'std3', 'loStd3', 'hiStd3', 
            'lotsize', 'optPrice', 'optMargin', 'rom']
    
    df_opt2 = df_opt2[cols]
    
    df_opt2.to_pickle(fspath+symbol+'.pkl')
    
    return None

In [ ]:
# upd_snps.py
from itertools import repeat
def upd_snps(ib, dfu):
    '''Updates the underlying snps
    Args:
       (ib) as connection object
       (dfu) as the underlying dataframe from p_snps
    Returns: None
       pickles back DataFrame with updated undPrice and margin'''

    # update prices
    tickers = ib.reqTickers(*dfu.contract)
    undPrices = {t.contract.symbol: t.marketPrice() for t in tickers} # {symbol: undPrice}

    # update margins - based on earliest expiration and strike closest to underlying price
    chains = {c.symbol: ib.reqSecDefOptParams(underlyingConId=c.conId, underlyingSecType=c.secType, underlyingSymbol=c.symbol, futFopExchange='')[0] for c in dfu.contract}

    lots_dict = dfu.lot.to_dict()

    order = Order(action='SELL', orderType='MKT', totalQuantity=100, whatIf=True)

    mdict = {i[0].symbol: int(pd.to_numeric(ib.whatIfOrder(i[0], i[1]).initMarginChange)) for i in zip((c for c in dfu.contract), repeat(order))}

    # updates
    dfu['undPrice'].update(pd.Series(undPrices))
    dfu['margin'].update(pd.Series(mdict))    

    # writeback
    dfu.to_pickle(fspath+'_lot_margin.pickle')
    
    return dfu

In [10]:
df_und = pd.read_pickle(fspath+'_lot_margin.pickle').dropna()

df_und = df_und.assign(und_remq=
                       (snp_assignment_limit/(df_und.lot*df_und.undPrice)).astype('int')) # remaining quantities in entire snp

# from portfolio
#_______________

p = util.df(ib.portfolio()) # portfolio table

# extract option contract info from portfolio table
dfp = pd.concat([p, util.df([c for c in p.contract])[util.df([c for c in p.contract]).columns[:7]]], axis=1).iloc[:, 1:]
dfp = dfp.rename(columns={'lastTradeDateOrContractMonth': 'expiration'})

# join the position with underlying contract details
dfp1 = dfp.set_index('symbol').join(df_und, how='left').drop(['contract'], axis=1)
dfp1 = dfp1.assign(qty=(dfp1.position/dfp1.lot).astype('int'))

dfp1.loc[dfp1.und_remq == 0, 'und_remq'] = 1   # for very large priced shares such as AMZN, BKNG, etc

# get the remaining quantities
remqty_dict = (dfp1.groupby(dfp1.index)['qty'].sum()+
               dfp1.groupby(dfp1.index)['und_remq'].mean()).to_dict()

remqty_dict = {k:(v if v > 0 else 0) for k, v in remqty_dict.items()} # portfolio's remq with negative values removed

df_und1 = df_und.assign(remqty=df_und.index.map(remqty_dict))
df_und1 = df_und1.assign(remqty=df_und1.remqty.fillna(df_und1.und_remq).astype('int'))

In [14]:
dfp1

,position,marketPrice,marketValue,averageCost,unrealizedPNL,realizedPNL,account,secType,conId,expiration,strike,right,multiplier,margin,undPrice,lot,und_remq,qty
ABBV,-5.0,0.793684,-396.84,123.627580,221.30,0.0,U8898867,OPT,358200413,20190503,75.5,P,100,-511,79.25,100,6,0
AMGN,-1.0,0.046487,-4.65,72.328300,67.68,0.0,U8898867,OPT,356149334,20190426,165.0,P,100,-5669,180.49,100,2,0
AMGN,-1.0,0.228745,-22.87,74.328200,51.45,0.0,U8898867,OPT,358205647,20190503,160.0,P,100,-5669,180.49,100,2,0
AMGN,-1.0,0.497531,-49.75,104.327800,54.57,0.0,U8898867,OPT,358205659,20190503,165.0,P,100,-5669,180.49,100,2,0
AMGN,-1.0,0.273958,-27.40,44.328600,16.93,0.0,U8898867,OPT,353820937,20190517,150.0,P,100,-5669,180.49,100,2,0
AMGN,-1.0,0.473637,-47.36,61.328400,13.96,0.0,U8898867,OPT,353820951,20190517,155.0,P,100,-5669,180.49,100,2,0
AMGN,-1.0,0.810000,-81.00,86.328100,5.33,0.0,U8898867,OPT,353820964,20190517,160.0,P,100,-5669,180.49,100,2,0
AMGN,-1.0,1.388903,-138.89,125.327600,-13.56,0.0,U8898867,OPT,353820974,20190517,165.0,P,100,-5669,180.49,100,2,0
BMY,-10.0,0.032214,-32.21,16.628980,134.08,0.0,U8898867,OPT,358219107,20190426,42.0,P,100,-274,45.35,100,11,0
BMY,-9.0,0.218533,-196.68,37.628711,141.98,0.0,U8898867,OPT,358219353,20190503,43.0,P,100,-274,45.35,100,11,0


In [4]:
# remqty_snp.py
import pandas as pd
from snp_variables import snp_assignment_limit, fspath

def remqty_snp(ib):
    '''generates the remaining quantities dictionary
    Args:
        (ib) as connection object
    Returns:
        remqty as a dictionary of {symbol: remqty}
        '''
    df_und = pd.read_pickle(fspath+'_lot_margin.pickle').dropna()

    df_und = df_und.assign(und_remq=
                           (snp_assignment_limit/(df_und.lot*df_und.undPrice)).astype('int')) # remaining quantities in entire snp

    # from portfolio
    #_______________

    p = util.df(ib.portfolio()) # portfolio table

    # extract option contract info from portfolio table
    dfp = pd.concat([p, util.df([c for c in p.contract])[util.df([c for c in p.contract]).columns[:7]]], axis=1).iloc[:, 1:]
    dfp = dfp.rename(columns={'lastTradeDateOrContractMonth': 'expiration'})

    # join the position with underlying contract details
    dfp1 = dfp.set_index('symbol').join(df_und, how='left').drop(['contract'], axis=1)
    dfp1 = dfp1.assign(qty=(dfp1.position/dfp1.lot).astype('int'))

    dfp1.loc[dfp1.und_remq == 0, 'und_remq'] = 1   # for very large priced shares such as AMZN, BKNG, etc

    # get the remaining quantities
    remqty_dict = (dfp1.groupby(dfp1.index)['qty'].sum()+
                   dfp1.groupby(dfp1.index)['und_remq'].mean()).to_dict()

    remqty_dict = {k:(v if v > 0 else 0) for k, v in remqty_dict.items()} # portfolio's remq with negative values removed

    df_und1 = df_und.assign(remqty=df_und.index.map(remqty_dict))
    df_und1 = df_und1.assign(remqty=df_und1.remqty.fillna(df_und1.und_remq).astype('int'))
    
    return df_und1

In [ ]:
%%time
p_snps(ib)

In [ ]:
%%time
p_snpopts(ib, u, p, l)

In [ ]:
import pandas as pd
dfu = pd.read_pickle(fspath+'_lot_margin.pickle')

In [ ]:
upd_snps(ib, dfu)

In [6]:
ib.portfolio()

[PortfolioItem(contract=Option(conId=358200413, symbol='ABBV', lastTradeDateOrContractMonth='20190503', strike=75.5, right='P', multiplier='100', primaryExchange='AMEX', currency='USD', localSymbol='ABBV  190503P00075500', tradingClass='ABBV'), position=-5.0, marketPrice=0.7985129, marketValue=-399.26, averageCost=123.62758, unrealizedPNL=218.88, realizedPNL=0.0, account='U8898867'),
 PortfolioItem(contract=Option(conId=356149334, symbol='AMGN', lastTradeDateOrContractMonth='20190426', strike=165.0, right='P', multiplier='100', primaryExchange='AMEX', currency='USD', localSymbol='AMGN  190426P00165000', tradingClass='AMGN'), position=-1.0, marketPrice=0.05116135, marketValue=-5.12, averageCost=72.3283, unrealizedPNL=67.21, realizedPNL=0.0, account='U8898867'),
 PortfolioItem(contract=Option(conId=358205647, symbol='AMGN', lastTradeDateOrContractMonth='20190503', strike=160.0, right='P', multiplier='100', primaryExchange='AMEX', currency='USD', localSymbol='AMGN  190503P00160000', tradi

In [9]:
remqty_snp(ib).loc['CVS']

contract    Stock(conId=2585769, symbol='CVS', exchange='S...
margin                                                  -1668
undPrice                                                53.07
lot                                                       100
und_remq                                                    9
remqty                                                      9
Name: CVS, dtype: object

In [ ]:
# update prices
tickers = ib.reqTickers(*dfu.contract)
undPrices = {t.contract.symbol: t.marketPrice() for t in tickers} # {symbol: undPrice}

# update margins - based on earliest expiration and strike closest to underlying price
chains = {c.symbol: ib.reqSecDefOptParams(underlyingConId=c.conId, underlyingSecType=c.secType, underlyingSymbol=c.symbol, futFopExchange='')[0] for c in dfu.contract}

lots_dict = dfu.lot.to_dict()

order = Order(action='SELL', orderType='MKT', totalQuantity=100, whatIf=True)

mdict = {i[0].symbol: int(pd.to_numeric(ib.whatIfOrder(i[0], i[1]).initMarginChange)) for i in zip((c for c in dfu.contract), repeat(order))}

# updates
dfu['undPrice'].update(pd.Series(undPrices))
dfu['margin'].update(pd.Series(mdict))